In [1]:
import os
import tensorflow as tf 
import numpy as np

In [2]:
# from imageio import imread, imsave
# import cv2
import os
root_dir='../dataset/scan'
in_dir='../dataset/scan/color/'
img_paths = [x.path for x in os.scandir(in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
num_img=len(img_paths)

In [3]:
from imageio import imread, imsave
photo = imread(img_paths[0])
photo.shape

(968, 1296, 3)

In [29]:
# 这些函数封装的真难用
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def write(input_file, output_file):
    img_paths = [x.path for x in os.scandir(in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
    num_img=len(img_paths)
    file_names = [(root_dir+'/color/{}.jpg'.format(f)).encode() for f in range(num_img)]
    writer = tf.python_io.TFRecordWriter(output_file) #定义writer，传入目标文件路径
    
    for i in range(num_img-15):
        rgb1_filename = (root_dir+'/color/'+"{}.jpg".format(i)).encode()
        rgb2_filename = (root_dir+'/color/'+"{}.jpg".format(i+10)).encode()
        depth1_filename = (root_dir+'/depth/'+"{}.jpg".format(i)).encode()
        depth2_filename = (root_dir+'/depth/'+"{}.jpg".format(i+10)).encode()
        shape1 = (968, 1296, 3)
        shape2 = (968, 1296, 3)
#             bbox1
#             bbox2
        c1Tw = np.loadtxt(root_dir+'/pose/'+"{}.txt".format(i),dtype=np.float32).ravel()
        c2Tw = np.loadtxt(root_dir+'/pose/'+"{}.txt".format(i+10),dtype=np.float32).ravel()
        K1 = np.loadtxt(root_dir+'/intrinsic/'+"intrinsic_color.txt",dtype=np.float32).ravel()
        K2 = np.loadtxt(root_dir+'/intrinsic/'+"intrinsic_color.txt",dtype=np.float32).ravel()
        tf_example = tf.train.Example(
            features=tf.train.Features(feature={
                'rgb1_filename': bytes_feature(rgb1_filename),
                'rgb2_filename': bytes_feature(rgb2_filename),
                'depth1_filename': bytes_feature(depth1_filename),
                'depth2_filename': bytes_feature(depth2_filename),
                'shape1': tf.train.Feature(int64_list=tf.train.Int64List(value=list(shape1))),
                'shape2': tf.train.Feature(int64_list=tf.train.Int64List(value=list(shape2))),
    #             'bbox1': int64_feature(), # x1,x2,y1,y2
    #             'bbox2': int64_feature(),
                'c1Tw': tf.train.Feature(float_list=tf.train.FloatList(value=c1Tw)),
                'c2Tw': tf.train.Feature(float_list=tf.train.FloatList(value=c2Tw)),
                'K1': tf.train.Feature(float_list=tf.train.FloatList(value=K1)),
                'K2': tf.train.Feature(float_list=tf.train.FloatList(value=K2)),
        }))

        #example序列化，并写入文件
        writer.write(tf_example.SerializeToString())
    writer.close()


In [30]:
write('.', 're.tfrecord')

In [7]:

def read_and_decode(file_name):
    filename_queue = tf.train.string_input_producer([file_name])
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                       features={"img_name": tf.FixedLenFeature([], tf.string),
                                                 "raw_img": tf.FixedLenFeature([], tf.string)})
    img_name = features["img_name"]
    image = tf.decode_raw(features['raw_img'], tf.uint8)
    image = tf.reshape(image, [256, 256, 3])
    return img_name, image


In [35]:
filename_queue = tf.train.string_input_producer(['re.tfrecord'])
reader = tf.TFRecordReader()

_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized_example,
            features={
                'rgb1_filename': tf.FixedLenFeature([], tf.string),
                'rgb2_filename': tf.FixedLenFeature([], tf.string),
                'depth1_filename': tf.FixedLenFeature([], tf.string),
                'depth2_filename': tf.FixedLenFeature([], tf.string),
                'shape1': tf.FixedLenFeature([3], tf.int64),
                'shape2': tf.FixedLenFeature([3], tf.int64),
#                 'bbox1': tf.FixedLenFeature([4], tf.int64), # x1,x2,y1,y2
#                 'bbox2': tf.FixedLenFeature([4], tf.int64),
                'c1Tw': tf.FixedLenFeature([16], tf.float32),
                'c2Tw': tf.FixedLenFeature([16], tf.float32),
                'K1': tf.FixedLenFeature([16], tf.float32),
                'K2': tf.FixedLenFeature([16], tf.float32),
                                                                          })

In [38]:

with tf.Session() as sess: #开始一个会话
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    coord=tf.train.Coordinator()
    threads= tf.train.start_queue_runners(coord=coord)
    x= sess.run(features['K1'])#在会话中取出image和label
    print(x)


[1.170188e+03 0.000000e+00 6.477500e+02 0.000000e+00 0.000000e+00
 1.170188e+03 4.837500e+02 0.000000e+00 0.000000e+00 0.000000e+00
 1.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 1.000000e+00]
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled
